In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import swifter
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from unidecode import unidecode
#from pprint import pprint

In [2]:
DATA_PATH = '../data'
RAW_DATA_PATH = DATA_PATH + '/raw'
CLEAN_DATA_PATH = DATA_PATH + '/clean'

In [3]:
le_monde_df = pd.read_csv(RAW_DATA_PATH + '/le_monde.csv', header=None)
le_monde_df

,0,1
0,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att..."
1,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...
2,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...
3,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult..."
4,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...
...,...,...
8561,https://www.lemonde.fr/international/article/2...,Retrait de Joe Biden : les réactions internati...
8562,https://www.lemonde.fr/culture/article/2024/07...,"A l’heure du bilan, le Festival « off » d’Avig..."
8563,https://www.lemonde.fr/series-d-ete/article/20...,"Les sneakers, partenaires de glisse du hip-hop"
8564,https://www.lemonde.fr/musiques/article/2024/0...,Nos choix de festivals et concerts : guitare e...


In [4]:
datafiles = []
for path in os.listdir(RAW_DATA_PATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(RAW_DATA_PATH, path)):
        datafiles.append(path)
print(datafiles)

['le_monde.csv', 'la_depeche.csv', 'actu_orange_insolite.csv', 'huffpost_insolite.csv', 'franceinfotv_bizarre.csv', 'europe1_insolite.csv', 'rfi.csv', 'demotivateur.csv', 'franceinfotv.csv', 'le_parisien.csv', 'liberation.csv', 'ouest_france.csv']


In [5]:
dfs = {}
for datafile in datafiles:
    dfs[datafile.replace('.csv', '')] = pd.read_csv(f'{RAW_DATA_PATH}/{datafile}', header=None)

In [6]:
dfs['liberation']

,0,1
0,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue..."
1,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...
2,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...
3,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...
4,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique"
...,...,...
5994,https://www.liberation.fr/international/ameriq...,«Kamala Harris part comme outsider face à Trum...
5995,https://www.liberation.fr/international/ameriq...,Tentative d’assassinat contre Trump : la direc...
5996,https://www.liberation.fr/sciences/decouverte-...,«Oxygène noir» : la molécule produite au fond ...
5997,https://www.liberation.fr/sports/jeux-olympiqu...,JO de Paris 2024 : l’escrimeuse Ysaora Thibus ...


In [7]:
for k, v in dfs.items():
    print(k, len(v))

le_monde 8566
la_depeche 49322
actu_orange_insolite 600
huffpost_insolite 10317
franceinfotv_bizarre 299
europe1_insolite 1704
rfi 7247
demotivateur 2381
franceinfotv 13977
le_parisien 17996
liberation 5999
ouest_france 100233


In [8]:
len(dfs['la_depeche']) // 3

16440

In [9]:
len(dfs['ouest_france']) // 6

16705

In [10]:
dfs['la_depeche'][:16440]

,0,1
0,https://www.ladepeche.fr/2024/10/19/bon-plan-b...,Bon plan bricolage : profitez du prix sacrifié...
1,https://www.ladepeche.fr/2024/10/19/enfin-de-r...,"Enfin de retour en stock, ce souffleur de feui..."
2,https://www.ladepeche.fr/2024/10/19/foncez-sur...,Foncez sur Cdiscount pour obtenir cette élague...
3,https://www.ladepeche.fr/2024/10/19/voici-la-m...,Voici la meilleure offre pour acheter le sécat...
4,https://www.ladepeche.fr/2024/10/19/bon-plan-a...,Bon plan aspirateur : 100 euros de remise immé...
...,...,...
16435,https://www.ladepeche.fr/2024/09/23/200-person...,200 personnes au repas du village
16436,https://www.ladepeche.fr/2024/09/23/de-ladour-...,De l’Adour à l’Arros
16437,https://www.ladepeche.fr/2024/09/23/du-cote-de...,Du côté de l’amicale loisirs et culture...
16438,https://www.ladepeche.fr/2024/09/23/festimages...,Fest’images : remise des portraits de l’expo


In [11]:
dfs['ouest_france'][:16705]

,0,1
0,https://www.ouest-france.fr/monde/gaza/33-mort...,33 morts dans une frappe israélienne sur un c...
1,https://www.ouest-france.fr/societe/faits-dive...,Policier blessé par arme à feu près de Nant...
2,https://www.ouest-france.fr/monde/japon/japon-...,Japon : un homme jette des cocktails molotov s...
3,https://www.ouest-france.fr/nouvelle-aquitaine...,Champdeniers-Saint-Denis. Aménagement de l’ac...
4,https://www.ouest-france.fr/nouvelle-aquitaine...,"Saint-Varent. Méthanisation, oui à l’extension"
...,...,...
16700,https://www.ouest-france.fr/pays-de-la-loire/t...,Le défi Foyers à alimentation positive d’Erd...
16701,https://www.ouest-france.fr/sport/football/red...,"Football. Le propriétaire du Red Star, 777 Pa..."
16702,https://www.ouest-france.fr/normandie/lisieux-...,"Plus d’un an après les émeutes à Lisieux, l..."
16703,https://www.ouest-france.fr/bretagne/finistere...,Le Sud-Finistère célèbre la culture urbaine...


In [12]:
dfs['ouest_france'][0][16704]

'https://www.ouest-france.fr/normandie/bretteville-sur-odon-14760/pres-de-caen-la-banque-alimentaire-recherche-des-benevoles-f6c3e4a6-840d-11ef-aa60-e9ecd5d09830'

In [13]:
dfs['la_depeche'] = dfs['la_depeche'][:16440]

In [14]:
dfs['ouest_france'] = dfs['ouest_france'][:16705]

In [15]:
for k, v in dfs.items():
    print(k, len(v))

le_monde 8566
la_depeche 16440
actu_orange_insolite 600
huffpost_insolite 10317
franceinfotv_bizarre 299
europe1_insolite 1704
rfi 7247
demotivateur 2381
franceinfotv 13977
le_parisien 17996
liberation 5999
ouest_france 16705


In [16]:
for k, v in dfs.items():
    v.insert(0, 'name', k)

In [17]:
dfs['le_monde']

,name,0,1
0,le_monde,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att..."
1,le_monde,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...
2,le_monde,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...
3,le_monde,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult..."
4,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...
...,...,...,...
8561,le_monde,https://www.lemonde.fr/international/article/2...,Retrait de Joe Biden : les réactions internati...
8562,le_monde,https://www.lemonde.fr/culture/article/2024/07...,"A l’heure du bilan, le Festival « off » d’Avig..."
8563,le_monde,https://www.lemonde.fr/series-d-ete/article/20...,"Les sneakers, partenaires de glisse du hip-hop"
8564,le_monde,https://www.lemonde.fr/musiques/article/2024/0...,Nos choix de festivals et concerts : guitare e...


In [18]:
dfs['le_monde'][~dfs['le_monde'][0].str.contains(r"/\d+/\d+/\d+/", regex=True)]

,name,0,1
852,le_monde,https://fait-pour-durer.lemonde.fr/les-jeux-ol...,"Les Jeux Olympiques et Paralympiques, créateur..."
854,le_monde,https://fait-pour-durer.lemonde.fr/apres-les-j...,Jeux Olympiques et Paralympiques : après l’ent...
855,le_monde,https://fait-pour-durer.lemonde.fr/apres-les-j...,"Après les Jeux, des transports pour tous !"
856,le_monde,https://fait-pour-durer.lemonde.fr/les-sites-o...,Les Sites Olympiques au service des Franciliens
857,le_monde,https://reinventez-la-mobilite-avec-audi.lemon...,Une certaine idée du mouvement
969,le_monde,https://grand-seiko.lemonde.fr/gardien-du-temp...,"Grand Seiko, gardien du temps et de la tradition"
1235,le_monde,https://ruinart-renouveau-adresse.lemonde.fr/,"Ruinart, le renouveau d’une adresse emblématique"
2376,le_monde,https://hurtigruten.lemonde.fr/,Les croisières Hurtigruten : se connecter au c...
2613,le_monde,https://vivez-montreal-en-hiver.lemonde.fr/le-...,"Montréal, le temps des gourmands"
2614,le_monde,https://vivez-montreal-en-hiver.lemonde.fr/un-...,"Montréal, un hiver éblouissant"


In [19]:
df = pd.concat([df for df in dfs.values()]).reset_index(drop=True)

In [20]:
df

,name,0,1
0,le_monde,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att..."
1,le_monde,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...
2,le_monde,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...
3,le_monde,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult..."
4,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...
...,...,...,...
102226,ouest_france,https://www.ouest-france.fr/pays-de-la-loire/t...,Le défi Foyers à alimentation positive d’Erd...
102227,ouest_france,https://www.ouest-france.fr/sport/football/red...,"Football. Le propriétaire du Red Star, 777 Pa..."
102228,ouest_france,https://www.ouest-france.fr/normandie/lisieux-...,"Plus d’un an après les émeutes à Lisieux, l..."
102229,ouest_france,https://www.ouest-france.fr/bretagne/finistere...,Le Sud-Finistère célèbre la culture urbaine...


In [21]:
df.columns = ['name', 'url', 'title']
df

,name,url,title
0,le_monde,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att..."
1,le_monde,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...
2,le_monde,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...
3,le_monde,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult..."
4,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...
...,...,...,...
102226,ouest_france,https://www.ouest-france.fr/pays-de-la-loire/t...,Le défi Foyers à alimentation positive d’Erd...
102227,ouest_france,https://www.ouest-france.fr/sport/football/red...,"Football. Le propriétaire du Red Star, 777 Pa..."
102228,ouest_france,https://www.ouest-france.fr/normandie/lisieux-...,"Plus d’un an après les émeutes à Lisieux, l..."
102229,ouest_france,https://www.ouest-france.fr/bretagne/finistere...,Le Sud-Finistère célèbre la culture urbaine...


In [22]:
df.name.unique()

array(['le_monde', 'la_depeche', 'actu_orange_insolite',
       'huffpost_insolite', 'franceinfotv_bizarre', 'europe1_insolite',
       'rfi', 'demotivateur', 'franceinfotv', 'le_parisien', 'liberation',
       'ouest_france'], dtype=object)

In [23]:
df[df.url.duplicated()]

,name,url,title
5550,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Les Etats-Unis sanctionnent des sociétés ayant...
26109,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un python birman de six mètres capturé en Fl...
26115,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un très rare « phallus de Titan » a fleuri en...
26164,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,"Pour incarner Henri VIII, Jude Law s’est mis a..."
26170,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,Que se passe-t-il avec les peintures de Pierre...
...,...,...,...
61438,franceinfotv,https://www.francetvinfo.fr/economie/pouvoir-d...,Pouvoir d’achat : les robes de mariage d'occas...
61496,franceinfotv,https://www.francetvinfo.fr/les-jeux-olympique...,"REPORTAGE. ""Cela fait un an qu'on organise les..."
61497,franceinfotv,https://www.francetvinfo.fr/monde/usa/presiden...,"RÉCIT. Débat raté, lapsus en série et retrait...."
61498,franceinfotv,https://www.francetvinfo.fr/sports/esport/info...,"INFO FRANCEINFO. Esport : l'EVO, plus grand év..."


In [24]:
for index, row in df[df.url.duplicated()].sort_values('url').iterrows():
    print(row['name'], row['url'])

franceinfotv https://www.francetvinfo.fr/alpes-maritimes/alpes-maritimes-des-attaques-de-loups-dans-un-village_6803107.html
franceinfotv https://www.francetvinfo.fr/coupe-du-monde/masculine/coupe-du-monde-de-futsal-l-equipe-de-france-se-hisse-en-demi-finale-en-battant-le-paraguay_6810403.html
franceinfotv https://www.francetvinfo.fr/culture/arts-expos/street-art/une-chevre-des-elephants-et-des-singes-des-enigmatiques-banksy-animaliers-fleurissent-a-londres_6714042.html
franceinfotv https://www.francetvinfo.fr/culture/cinema/alain-delon/mort-d-alain-delon-de-romy-schneider-a-mireille-darc-qui-furent-les-femmes-de-sa-vie-sans-qui-il-n-aurait-ete-que-l-ombre-de-l-acteur-et-de-l-homme_3702783.html
franceinfotv https://www.francetvinfo.fr/culture/cinema/alain-delon/mort-d-alain-delon-film-comique-cap-de-charcuterie-l-homme-meconnu_6732897.html
franceinfotv https://www.francetvinfo.fr/culture/cinema/l-equipe-du-film-l-abbe-pierre-une-vie-de-combats-condamne-les-agissements-du-religieux-et-ap

In [25]:
df[df.url=='https://www.francetvinfo.fr/decouverte/bizarre/le-camping-des-eurockeennes-elit-son-maire_1620667.html']

,name,url,title
36131,franceinfotv_bizarre,https://www.francetvinfo.fr/decouverte/bizarre...,Le camping des Eurockéennes élit son maire
36132,franceinfotv_bizarre,https://www.francetvinfo.fr/decouverte/bizarre...,Le camping des Eurockéennes élit son maire


In [26]:
df = df.drop_duplicates('url').reset_index(drop=True)
df

,name,url,title
0,le_monde,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att..."
1,le_monde,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...
2,le_monde,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...
3,le_monde,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult..."
4,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...
...,...,...,...
102080,ouest_france,https://www.ouest-france.fr/pays-de-la-loire/t...,Le défi Foyers à alimentation positive d’Erd...
102081,ouest_france,https://www.ouest-france.fr/sport/football/red...,"Football. Le propriétaire du Red Star, 777 Pa..."
102082,ouest_france,https://www.ouest-france.fr/normandie/lisieux-...,"Plus d’un an après les émeutes à Lisieux, l..."
102083,ouest_france,https://www.ouest-france.fr/bretagne/finistere...,Le Sud-Finistère célèbre la culture urbaine...


In [27]:
nlp = spacy.load("fr_core_news_md")

In [28]:
STOP_WORDS

{'a',
 'abord',
 'afin',
 'ah',
 'ai',
 'aie',
 'ainsi',
 'ait',
 'allaient',
 'allons',
 'alors',
 'anterieur',
 'anterieure',
 'anterieures',
 'antérieur',
 'antérieure',
 'antérieures',
 'apres',
 'après',
 'as',
 'assez',
 'attendu',
 'au',
 'aupres',
 'auquel',
 'aura',
 'auraient',
 'aurait',
 'auront',
 'aussi',
 'autre',
 'autrement',
 'autres',
 'autrui',
 'aux',
 'auxquelles',
 'auxquels',
 'avaient',
 'avais',
 'avait',
 'avant',
 'avec',
 'avoir',
 'avons',
 'ayant',
 'bas',
 'basee',
 'bat',
 "c'",
 'car',
 'ce',
 'ceci',
 'cela',
 'celle',
 'celle-ci',
 'celle-la',
 'celle-là',
 'celles',
 'celles-ci',
 'celles-la',
 'celles-là',
 'celui',
 'celui-ci',
 'celui-la',
 'celui-là',
 'cent',
 'cependant',
 'certain',
 'certaine',
 'certaines',
 'certains',
 'certes',
 'ces',
 'cet',
 'cette',
 'ceux',
 'ceux-ci',
 'ceux-là',
 'chacun',
 'chacune',
 'chaque',
 'chez',
 'ci',
 'cinq',
 'cinquantaine',
 'cinquante',
 'cinquantième',
 'cinquième',
 'combien',
 'comme',
 'comment',

In [29]:
def lemmatize(s: str) -> iter:
    s = re.sub(r'[():]', ' ', s)
    # tokenize
    doc = nlp(s)

    # remove punct and stopwords
    tokens = filter(lambda token: not token.is_space and not token.is_punct and not token.is_stop and not token.is_digit \
        #and str(token) not in STOP_WORDS \
        and unidecode(str(token)) not in ['a', 'd\'1', '-t'] \
        and not re.search(r'(^\d+[,mh/]\d+$)|(^\d+[eè]me$)|(^\d+[ehk]$)|\d+.\d{3}$|[jJ]\-\d+$|^\-?\d+$|^s\d+e\d+$|^\d+/\d+/\d+$|^\-', str(token), re.IGNORECASE), doc)

    tokens = map(lambda token: unidecode(token.lemma_.lower()), tokens)

    tokens = map(lambda token: re.sub(r'[\[\]<>()"]', '', token), tokens)
    
    tokens = map(lambda token: re.sub(r'[.\'\-,/]$', '', token), tokens)

    tokens = map(lambda token: re.sub(r'^[.\'/]', '', token), tokens)

    tokens = map(lambda token: re.sub(r'^.\'', '', token), tokens)

    tokens = map(lambda token: re.sub(r'\*.*$', '', token), tokens)

    tokens = map(lambda token: re.sub(r'^\d+[,mh/]\d+$', '', token), tokens)

    tokens = filter(lambda token: not re.search(r'^[0-9]+(eme|mn|er|es|e)$', token), tokens)

    tokens = filter(lambda token: not re.search(r'^\-', token), tokens)

    tokens = filter(lambda token: not re.search(r'^\d+h/\d+h$', token), tokens)

    tokens = filter(lambda token: not re.search(r'[\[\]<>]', token), tokens)

    tokens = filter(lambda token: len(token) > 1, tokens)

    #tokens = filter(lambda token: token not in STOP_WORDS, tokens)
    
    return tokens







In [30]:
sentences = [
    "Silence on joue ! «Caravan SandWitch», «Warhammer 40,000 : Space Marine 2», «Riven», «Eté»",
    "Les émissions de gaz à effet de serre ont baissé de 3,6 % en France au premier semestre 2024",
    "Polar : Jo Nesbø, «pulp» et fiction",
    "Les femmes esclaves des diplomates (4/4) : à Paris, la fuite et la double peine d’une domestique de l’ex-ambassadeur du Chili auprès de l’OCDE",
    "Ce monstrueux poisson chat de 2m85 pêché après un combat de 43 minutes pourrait établir un nouveau record",
    "Fatigué d'être un humain, un Japonais dépense plus de 21.000 euros pour... devenir un loup",
    "Pays basque : quatre amis landais ont gravi la Rhune en 1h27... perchés sur des échasses, une prouesse !",
    "États-Unis : cette grand-mère a fêté son 100ème anniversaire le jour des 1 an de son arrière-arrière-petite-fille",
    "Le robot aspirateur de cet hôtel à Cambridge retrouvé 24h après sa fuite",
    "Le streamer Arkunir récolte 47.500 euros pour \"30 millions d'amis\"",
    "Elections américaines, J-18 : Kamala Harris face à Fox News, DJ Trump face aux indécis, les sondages figés et la ruée aux urnes bien entamée",
    "Corée du Sud : Séoul enregistre sa 26e «nuit tropicale» d’affilée, la plus longue série depuis 118 ans",
    "Disparition de Medhi Narjissi en Afrique du sud : l’encadrement de l’équipe de France U18 de rugby suspendu",
    "\"Ralph 2.0\": la fillette de ce célèbre mème a bien grandi",
    "Manger des pâtes par -65°C peut s'avérer compliqué",
    "Cinq passages piétons avec effet 3D expérimentés à Paris",
    "A New York, la mairie veut rendre la Ve Avenue aux piétons",
    "My Million : Ce Français avait une chance sur 16.000 milliards de décrocher ce double pactole (et il l'a fait)",
    "Cet enfant a découvert un fossile de stegomastodon né 1.199.991 ans avant lui",
    "Rémi Gaillard va être filmé 24h/24 dans une cage de la SPA",
    "Élan:1 - Tondeuse à gazon:0",
    "Dans le 13h de TF1, Jean-Pierre Pernaut se prend pour Joséphine ange gardien",
    "PHOTOS. Quand les stormtroopers (et R2-D2) de Star Wars débarquent à la Maison Blanche",
    "Game of Thrones s05e01 : le résumé de l'épisode 1 de la saison 5 [ATTENTION SPOILERS]",
    "VIDÉO. L'affaire Nabilla résumée façon jeu vidéo 8-bit",
    "VIDÉO. Vivre dans une chambre de bonne à Paris: l'ingénieux placard d'un studio de 8m2",
    "PHOTOS. Ronaldo fête ses sous-vêtements : il mange un gâteau à l'effigie de son caleçon pour le premier anniversaire de sa marque CR7 Underwear",
    "TÉMOIGNAGE. 80e D-Day. En octobre 1944, reprise des liaisons en car Argentan-Domfront",
    "Bon plan pour les randonneurs : -45 % sur ce modèle adidas mi-montante et étanche",
    "Près d’1 million de Normands vivent ailleurs qu’en Normandie : enquête sur la diaspora normande",
    "Basket - N1M. Frédéric Loubaki a déjà tout pigé avec les Béliers de Kemper",
    "PowerPoint, Word, Excel... Toute la suite Office à moins d’1 euro à perpétuité sur votre ordinateur",
    "Journal de 7H45 : l'opération Brioches avec l'ADAPEI 44",
    "« Bonjour ». Karima Charni reviendra-t-elle dans l’émission de TF1 après la « Star Academy » ?",
    "Voici un 2ème code promo : moins de 165 euros pour le Xiaomi Redmi Note 13 Pro 5G ce mercredi",
    "Jusqu’à 300mn d’autonomie pour cet aspirateur robot qui ne laisse aucun déchet sur son passage",
    "VIDÉO. 7-Octobre : début des cérémonies en Israël sur le site du festival Nova",
    "Une bière du 4ème siècle recréée au Canada",
    "Il promet 3.000 litres de lait pour les Restos du Cœur si sa vidéo est vue 10.000 fois",
    "Le tramadol et la codéine seront prescrits sur ordonnance infalsifiable à partir du 1ᵉʳ décembre",
    "Eau : \"À l’horizon 2040/2050, la baisse des ressources est estimée entre 20 et 40 % sur le territoire\" de Tarbes-Lourdes-Pyrénées",
    "le 21h/22h du vendredi 18 octobre 2024",
    "\"20h30 le samedi\". La story Amélie Nothomb",
    "Bulletin météo du samedi 12 octobre 2024 à 20h53",
    "19/20 info du jeudi 22 août 2024",
    "Cette illusion d'optique sur un chat a amusé 90k d'internautes sur Twitter",
    "Nés entre 22h et 23h le 22/02/2022, ces jumeaux ont gagné un forfait de 22 ans de ski à la station des 2 Alpes",
    "Ambiance « Bal Tropical » pour les 37es Nuits de Champagne à Troyes",
    "Vuelta, 18e étape : O’Connor toujours en rouge, David Gaudu se hisse dans le top 5",
    "« Départ 18:25 » : comment bénéficier de l’aide de 250 euros proposée aux jeunes pour partir en vacances",
    "IA : utiliser ChatGPT comme psy, est-ce une bonne idée ?",
    "Silence on joue ! Ubisoft dans la tourmente, «le Vaillant Petit Page», «Thank Goodness You’re Here»",
    "P. Diddy, l’ascension monstre d’un magnat du hip-hop",
    "Election présidentielle américaine, J- 56 : Harris et Trump sur un ring",
    "Paris 2024 : en bronze sur 400 m, Guillaume Junior Atangana offre la seconde médaille de l’histoire à l’équipe paralympique de réfugiés",
    "«J’ai aimé cette vie-là» : au camping de Dol-de-Bretagne, les derniers mois d’un paradis précaire",
    "Au meeting de Kamala Harris à Atlanta, l’espoir d’«un avenir radieux» électrise les militants démocrates",
    "Près de 200 personnes secourues en quelques heures par l’«Ocean Viking» au large de la Libye",
    "Une expo Lego® dans cet hypermarché de Challans",
    "Le Test n'est pas (encore) concluant", # personal test
    "Le développement est (re)commencé",    # personal test
    "Val d'Anast. Du nouveau pour Val-d’Anast Motoculture et loisirs",
    "QUIZ. « Hélène et les garçons », « Les mystères de l’amour » : connaissez-vous bien Hélène Rollès ?",
    "« Un p’tit truc en plus » cartonne en salles et... en cuisine à Mulhouse",
    "Découpez jusqu’à 1000 bûches par heure avec cette fendeuse à bois électrique à prix cassé",
    "Intempéries : un mois de pluie en 24h !",
    "Moteur ! Ca tourne tout le week-end pour le Festival Alain-Resnais",
    "« La tech rêve d’“agents”, des IA capables de planifier et d’agir »",
    "Impulscience®, un programme pour répondre à l’urgence scientifique",
    "Est-ce que l’électricité « verte » est vraiment « verte » ?",
    "J. D. Vance, le colistier de Trump, à droite toute",
    "« Paris, musée du XXIᵉ siècle. Le 18ᵉ arrondissement », de Thomas Clerc : le feuilleton littéraire de Tiphaine Samoyault",
    "Un apéro avec Cédric Doumbè : « Je mets tout le monde K.-O., c’est un de mes défauts »",
    "Jeunes salarié·es, vous habitez toujours chez vos parents ? Racontez-nous",
    "« Bergers en famille », sur Arte.tv : une vie à la montagne en symbiose avec les bêtes",
    "Boeing suspend les vols tests du 777X après la défaillance d’une pièce",
    "« L’Aventure du câble transatlantique. Une révolution du XIXᵉ siècle », sur Arte : relier le Vieux Continent au Nouveau Monde, un fabuleux défi technologique",
    "Elon Musk annonce la fermeture des bureaux de X au Brésil et s’en prend au juge de la Cour suprême brésilienne, Alexandre de Moraes",
    "M & M’s achète Pringles et Kellog’s pour 36 milliards de dollars",
    "La 20e édition de la ronde des Toupies",
    "Le retour de la Rur@linette dans  le cœur du bourg",
    "Ça y est, la 4 L roule de nouveau parmi nous !",
    "Déviances sexuelles, l’Eglise doit redevenir une maison sûre",
    "Les enfants du Patrimoine reçus au musée A. Bergès",
    "Résultats de l’U.S.C.V Foot",
    "Nettoyeur haute pression Kärcher : ce modèle de 1800W facile à utiliser voit son prix chuter",
    "Le club d’athlétisme relève le challenge Run2K",
    "D’une info à l’autre",
    "Intégration des 6es de St Jo à Graves",
    "Bulletin météo du samedi 19 octobre 2024 à 13h55",
    "Le 16h/18h du vendredi 18 octobre 2024",
    "Evènement : La 31ème édition du Prix Bayeux Calvados-Normandie des correspondants de guerre, du 7 au 13 octobre 2024.",
    "L'\"Odyssey\" va pouvoir débuter sa croisière de trois ans et demi autour du monde, après être resté bloqué quatre mois à Belfast",
    "La Russie ne s'«inquiète pas» d’une arrestation de Vladimir Poutine durant sa visite en Mongolie, membre de la CPI",
    "McDonald's ferme en Russie, les habitants congèlent des burgers et les revendent à des prix exorbitants",
    "VIDÉO. « C’était un cauchemar » : à Paris, la Cité Rose pleure ce père de famille tué par la chute d’un arbre",
    "« Il y a un train à prendre, on veut y monter » : les joueurs du Paris FC motivés par le futur rachat du club",
    "M&M’s : Rouge, Bleu ou Jaune, des mascottes à l’image de la société",
    "Marathon pour tous : 30 °C à 21 heures, humidité... dans quelles conditions météo la course aura-t-elle lieu ?",
    "La rave party touche à sa fin à Château-Gontier,﻿ les derniers teufeurs quittent le site", # comma at the end,
    "SONDAGE. Federer, Nadal,﻿﻿ Sampras, Djokovic... Qui est le meilleur joueur de l’histoire du tennis ?", # comma at the end,
    "Une huître géante d'1,4 kilo découverte en Vendée", # a number with comma,
    "Moschino vend une robe \"housse de pressing en plastique\" à près de 600€. Tout simplement",
    "Alfred Hitchcock ou Charlie Chaplin peut-être bientôt sur les billets de £20",
    "Retrait de Joe Biden : vers un duel Kamala Harris/ Donald Trump ?",
    "Échanges Donald Trump /Elon Musk sur X: «Il y a une crainte des États-Unis à pratiquer une régulation des réseaux sociaux»",
    "Job de rêve : devenez testeur de matelas professionnel et soyez payé 1600 €/mois pour... dormir !",
]

# search remaining flaws in dataset with regular expression : [^a-z0-9.\-+\s']

for sentence in sentences:
    print(list(lemmatize(sentence)))















['silence', 'joue', 'caravan', 'sandwitch', 'warhammer', 'space', 'marine', 'riven', 'ete']
['emission', 'gaz', 'serre', 'baisser', 'france', 'semestre']
['polar', 'jo', 'nesbo', 'pulp', 'fiction']
['femme', 'esclave', 'diplomate', 'paris', 'fuite', 'double', 'peine', 'domestique', 'ex-ambassadeur', 'chili', 'aupres', 'ocde']
['monstrueu', 'poisson', 'chat', 'pecher', 'combat', 'minute', 'etablir', 'record']
['fatigue', 'humain', 'japonais', 'depense', 'euro', 'devenir', 'loup']
['pays', 'basque', 'ami', 'landais', 'gravir', 'rhune', 'perche', 'echasse', 'prouesse']
['etats-unis', 'grand-mere', 'feter', 'anniversaire', 'jour', 'an', 'arriere-arriere-petite-fille']
['robot', 'aspirateur', 'hotel', 'cambridge', 'retrouve', 'apres', 'fuite']
['streamer', 'arkunir', 'recolte', 'euro', 'million', 'ami']
['election', 'americain', 'kamala', 'harris', 'face', 'fox', 'news', 'dj', 'trump', 'face', 'indecis', 'sondage', 'figer', 'ruee', 'urne', 'bien', 'entamer']
['coree', 'sud', 'seoul', 'enreg

In [31]:
df['lemmes'] = df.title.swifter.apply(lambda text: ' '.join(map(str, lemmatize(text))))

Pandas Apply:   0%|          | 0/102085 [00:00<?, ?it/s]

In [32]:
# search remaining flaws in dataset with regular expression : [^a-z0-9.\-+\s']
#dfs['le_monde'][~dfs['le_monde'][0].str.contains(r"/\d+/\d+/\d+/", regex=True)]
spec_chars_lemmes = df[df['lemmes'].str.contains(r"[^a-z0-9.\-+\s']", regex=True)]['lemmes']
print(len(spec_chars_lemmes))
for lemme in spec_chars_lemmes:
    print(lemme)

68
doliprane fonds pai partners surencherir americain cd&r racheter filiale sanofi
revue usbek rica changer nom devenir futu&r
sanofi vendre opella produire doliprane fonds americain cd&r milliard euro
comete c/2023 a3 tsuchinshan atlas approche bientot visible europe
festival cinema americain deauville mue post-#metoo
mostra venise thriller erotique babygirl jeter pave cinema post-#metoo
revanche sorciere post-#metoo rituel moment precieux arreter monde acceleration
agence s&p abaisse note ukraine cran defaut paiement
rur@linette coeur bourg
budget departement amputer mEUR
coupe budgetaire mEUR pouvoir prelever departement lot-et-garonne etat
renaissance vert place providence 1mEUR officiellement inaugurer
coupe budgetaire trou mEUR toulouse metropole jean-luc moudenc annoncer gel embauche
responsable le&c grand sud reunir
responsable le&c grand sud reunir cazeres
braderie brad'EURco affaire vue
info depeche re-sourcer fonds investissement toulousain venir lever mEUR
festival locombia

In [33]:
df['funny'] = 0
df

,name,url,title,lemmes,funny
0,le_monde,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att...",simon fieschi grievement blesser attentat cont...,0
1,le_monde,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...,falloir expliquer difference effet concret imm...,0
2,le_monde,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...,peripherique outil evaluateur kilometre heure,0
3,le_monde,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult...",occitanie agriculteur mecontent multiplier act...,0
4,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...,presidentielle americain donald trump instrume...,0
...,...,...,...,...,...
102080,ouest_france,https://www.ouest-france.fr/pays-de-la-loire/t...,Le défi Foyers à alimentation positive d’Erd...,defi foyers alimentation positif erdre gesvre ...,0
102081,ouest_france,https://www.ouest-france.fr/sport/football/red...,"Football. Le propriétaire du Red Star, 777 Pa...",football proprietaire red star partners failli...,0
102082,ouest_france,https://www.ouest-france.fr/normandie/lisieux-...,"Plus d’un an après les émeutes à Lisieux, l...",an apres emeute lisieux poste police municipal...,0
102083,ouest_france,https://www.ouest-france.fr/bretagne/finistere...,Le Sud-Finistère célèbre la culture urbaine...,sud finistere celebre culture urbain etat,0


In [34]:
df[df['name'].isin(['actu_orange_insolite', 'demotivateur', 'europe1_insolite', 'franceinfotv_bizarre', 'huffpost_insolite'])]

,name,url,title,lemmes,funny
25005,actu_orange_insolite,https://actu.orange.fr/societe/insolite/barbie...,"Barbier, chasseur et bouledogue anglais: les i...",barbier chasseur bouledogue anglais insolite a...,0
25006,actu_orange_insolite,https://actu.orange.fr/societe/insolite/etats-...,États-Unis : rappelé au travail pendant ses co...,etats-unis rappeler travail conge devenir mill...,0
25007,actu_orange_insolite,https://actu.orange.fr/societe/insolite/rassur...,"""Rassurez-vous"" : la compagnie ferroviaire Tre...",rassurer compagnie ferroviaire trenitalia cham...,0
25008,actu_orange_insolite,https://actu.orange.fr/societe/insolite/un-rap...,"""Un rapport qualité prix imbattable"" : un homm...",rapport qualite prix imbattable homme sillonne...,0
25009,actu_orange_insolite,https://actu.orange.fr/societe/insolite/washin...,"Washington accueille deux nouveaux ""pandassade...",washington accueillir nouveau pandassadeur chi...,0
...,...,...,...,...,...
47537,demotivateur,https://www.demotivateur.fr/insolite/pour-l-ou...,"À Détroit, Eminem assure lui-même le service p...",detroit eminem assurer service ouverture resta...,0
47538,demotivateur,https://www.demotivateur.fr/insolite/scandinav...,Scandinavie : s'enfoncer des sachets de tabac ...,scandinavie enfoncer sachet tabac prepuce anus...,0
47539,demotivateur,https://www.demotivateur.fr/insolite/las-vegas...,"Découvrez les premières images de « Moon », l'...",decouvrir premier image moon incroyable hotel ...,0
47540,demotivateur,https://www.demotivateur.fr/insolite/ivre-et-a...,"Ivre et porté disparu, il est retrouvé par has...",ivre porter disparaitre retrouver hasard parti...,0


In [35]:
df.loc[df['name'].isin(['actu_orange_insolite', 'demotivateur', 'europe1_insolite', 'franceinfotv_bizarre', 'huffpost_insolite']), 'funny'] = 1

In [36]:
df[df['name'].isin(['actu_orange_insolite', 'demotivateur', 'europe1_insolite', 'franceinfotv_bizarre', 'huffpost_insolite'])]

,name,url,title,lemmes,funny
25005,actu_orange_insolite,https://actu.orange.fr/societe/insolite/barbie...,"Barbier, chasseur et bouledogue anglais: les i...",barbier chasseur bouledogue anglais insolite a...,1
25006,actu_orange_insolite,https://actu.orange.fr/societe/insolite/etats-...,États-Unis : rappelé au travail pendant ses co...,etats-unis rappeler travail conge devenir mill...,1
25007,actu_orange_insolite,https://actu.orange.fr/societe/insolite/rassur...,"""Rassurez-vous"" : la compagnie ferroviaire Tre...",rassurer compagnie ferroviaire trenitalia cham...,1
25008,actu_orange_insolite,https://actu.orange.fr/societe/insolite/un-rap...,"""Un rapport qualité prix imbattable"" : un homm...",rapport qualite prix imbattable homme sillonne...,1
25009,actu_orange_insolite,https://actu.orange.fr/societe/insolite/washin...,"Washington accueille deux nouveaux ""pandassade...",washington accueillir nouveau pandassadeur chi...,1
...,...,...,...,...,...
47537,demotivateur,https://www.demotivateur.fr/insolite/pour-l-ou...,"À Détroit, Eminem assure lui-même le service p...",detroit eminem assurer service ouverture resta...,1
47538,demotivateur,https://www.demotivateur.fr/insolite/scandinav...,Scandinavie : s'enfoncer des sachets de tabac ...,scandinavie enfoncer sachet tabac prepuce anus...,1
47539,demotivateur,https://www.demotivateur.fr/insolite/las-vegas...,"Découvrez les premières images de « Moon », l'...",decouvrir premier image moon incroyable hotel ...,1
47540,demotivateur,https://www.demotivateur.fr/insolite/ivre-et-a...,"Ivre et porté disparu, il est retrouvé par has...",ivre porter disparaitre retrouver hasard parti...,1


In [37]:
df[df['funny'] == 1].name.unique()

array(['actu_orange_insolite', 'huffpost_insolite',
       'franceinfotv_bizarre', 'europe1_insolite', 'demotivateur'],
      dtype=object)

In [38]:
df[df['funny'] == 0].name.unique()

array(['le_monde', 'la_depeche', 'rfi', 'franceinfotv', 'le_parisien',
       'liberation', 'ouest_france'], dtype=object)

In [39]:
df[df['name'].str.contains('_insolite')]

,name,url,title,lemmes,funny
25005,actu_orange_insolite,https://actu.orange.fr/societe/insolite/barbie...,"Barbier, chasseur et bouledogue anglais: les i...",barbier chasseur bouledogue anglais insolite a...,1
25006,actu_orange_insolite,https://actu.orange.fr/societe/insolite/etats-...,États-Unis : rappelé au travail pendant ses co...,etats-unis rappeler travail conge devenir mill...,1
25007,actu_orange_insolite,https://actu.orange.fr/societe/insolite/rassur...,"""Rassurez-vous"" : la compagnie ferroviaire Tre...",rassurer compagnie ferroviaire trenitalia cham...,1
25008,actu_orange_insolite,https://actu.orange.fr/societe/insolite/un-rap...,"""Un rapport qualité prix imbattable"" : un homm...",rapport qualite prix imbattable homme sillonne...,1
25009,actu_orange_insolite,https://actu.orange.fr/societe/insolite/washin...,"Washington accueille deux nouveaux ""pandassade...",washington accueillir nouveau pandassadeur chi...,1
...,...,...,...,...,...
37909,europe1_insolite,https://www.europe1.fr/insolite/La-Thailande-s...,La Thaïlande sauve 750 chiens de la cuisine,thailande sauf chien cuisine,1
37910,europe1_insolite,https://www.europe1.fr/insolite/Elle-transport...,Elle transportait de la cocaïne dans ses seins,transporter cocaine sein,1
37911,europe1_insolite,https://www.europe1.fr/insolite/Intouchables-e...,Intouchables : elle accouche presque de rire,intouchable accouche presque rire,1
37912,europe1_insolite,https://www.europe1.fr/insolite/GB-un-cambriol...,GB : un cambrioleur insulte ses victimes,gb cambrioleur insulter victime,1


In [40]:
df['name'] = df['name'].str.replace('_insolite', '')

In [41]:
df['name'] = df['name'].str.replace('_bizarre', '')

In [42]:
df

,name,url,title,lemmes,funny
0,le_monde,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att...",simon fieschi grievement blesser attentat cont...,0
1,le_monde,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...,falloir expliquer difference effet concret imm...,0
2,le_monde,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...,peripherique outil evaluateur kilometre heure,0
3,le_monde,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult...",occitanie agriculteur mecontent multiplier act...,0
4,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...,presidentielle americain donald trump instrume...,0
...,...,...,...,...,...
102080,ouest_france,https://www.ouest-france.fr/pays-de-la-loire/t...,Le défi Foyers à alimentation positive d’Erd...,defi foyers alimentation positif erdre gesvre ...,0
102081,ouest_france,https://www.ouest-france.fr/sport/football/red...,"Football. Le propriétaire du Red Star, 777 Pa...",football proprietaire red star partners failli...,0
102082,ouest_france,https://www.ouest-france.fr/normandie/lisieux-...,"Plus d’un an après les émeutes à Lisieux, l...",an apres emeute lisieux poste police municipal...,0
102083,ouest_france,https://www.ouest-france.fr/bretagne/finistere...,Le Sud-Finistère célèbre la culture urbaine...,sud finistere celebre culture urbain etat,0


In [43]:
df.name.unique()

array(['le_monde', 'la_depeche', 'actu_orange', 'huffpost',
       'franceinfotv', 'europe1', 'rfi', 'demotivateur', 'le_parisien',
       'liberation', 'ouest_france'], dtype=object)

In [44]:
print(df[['name', 'funny']].drop_duplicates().sort_values('funny').to_string(index=False))

        name  funny
    le_monde      0
  la_depeche      0
         rfi      0
franceinfotv      0
 le_parisien      0
  liberation      0
ouest_france      0
 actu_orange      1
    huffpost      1
franceinfotv      1
     europe1      1
demotivateur      1


In [45]:
df.to_csv(CLEAN_DATA_PATH + '/dataset.csv', index=False)